# This is a notebook to generate Deep mutation scanning library for scoring sequence using ESM_if based on protein backbone

To order to compare computational and expeirmental results,
we need to match sequences
2. We need use mask in cooridnates for sites where pdb cooridnates are not avaialale

The following is an application to SHP2 protein.

In [1]:
import esm
#load the model
model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
model = model.eval()  

/home/yunyao/miniconda3/envs/struct-evo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yunyao/miniconda3/envs/struct-evo/lib/python3.9/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [2]:
# information on structure to be designed
fpath = './4DGP.pdb' # .pdb format is also acceptable
chain_id = 'A'
structure = esm.inverse_folding.util.load_structure(fpath, chain_id)
coords, native_seq = esm.inverse_folding.util.extract_coords_from_structure(structure)
print('Native sequence:')
print(native_seq)
print('Coordinates shape:', coords.shape)


Native sequence:
SRRWFHPNITGVEAENLLLTRGVDGSFLARPSKSNPGDFTLSVRRNGAVTHIKIQNTGDYYDLYGGEKFATLAELVQYYMEHHGQLKEKNGDVIELKYPLNCADPTSERWFHGHLSGKEAEKLLTEKGKHGSFLVRESQSHPGDFVLSVRTGDDKGESNDGKSKVTHVMIRCQELKYDVGGGERFDSLTDLVEHYKKNPMVETLGTVLQLKQPLNTTRINAAEIESRVRELSKGFWEEFETLQQQECKLLYSRKEGQRQENKNKNRYKNILPFDHTRVVLHDSDYINANIIMPEKKSYIATQGCLQNTVNDFWRMVFQENSRVIVMTTKEVERGKSKCVKYWPDEYALKEYGVMRVRNVKESAAHDYTLRELKLSKVGQGNTERTVWQYHFRTWPDHGVPSDPGGVLDFLEEVHHKQESIMDAGPVVVHCSAGIGRTGTFIVIDILIDIIREKGVDCDIDVPKTIQMVRSQRSGMVQTEAQYRFIYMAVQHYIETLQRR
Coordinates shape: (499, 3, 3)


##  Warning!! The above native sequence is not correct.

The model ignores residue of which coordiantes are not available.

In [3]:
import os
import sys
dir_path = '/home/yunyao/structural-evolution/bin'

# Add the directory containing the modules to sys.path
if dir_path not in sys.path:
    sys.path.append(dir_path)

In [23]:
# we can test the fitnesss of various sequences generated by dms

import argparse
from dms_utils import deep_mutational_scan
from pathlib import Path
import numpy as np

import esm
from util import load_structure, extract_coords_from_structure
import biotite.structure
from collections import defaultdict


def get_native_seq(pdbfile, chain):
    structure = load_structure(pdbfile, chain)
    _ , native_seq = extract_coords_from_structure(structure)
    return native_seq


def read_fasta(file_path):
    '''Reads a FASTA file and returns a dictionary with sequence IDs as keys and sequences as values.'''
    sequences = {}
    with open(file_path, 'r') as f:
        seq_id = None
        seq = ''
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if seq_id is not None:
                    sequences[seq_id] = seq
                seq_id = line[1:5]  # Remove '>'
                seq = ''
            else:
                seq += line
        if seq_id is not None:
            sequences[seq_id] = seq  # Add the last sequence
    return seq
    
def write_dms_lib(args):
    '''Writes a deep mutational scanning library, including the native/wildtype (wt) of the 
    indicated target chain in the structure to an output Fasta file'''

    sequence = ''.join(read_fasta(args.seqfile).split())
    Path(args.dmspath).parent.mkdir(parents=True, exist_ok=True)
    with open(args.dmspath, 'w') as f:
        f.write('>wt\n')
        f.write(sequence+'\n')
        for pos, wt, mt in deep_mutational_scan(sequence):
            assert(sequence[pos] == wt)
            mut_seq = sequence[:pos] + mt + sequence[(pos + 1):]

            f.write('>' + str(wt) + str(pos+1) + str(mt) + '\n')
            f.write(mut_seq + '\n')



In [ ]:
# test the read_fasta function
test_seqs= read_fasta('/home/yunyao/SH2/6CRF.fasta')
test_seq


'GMTSRRWFHPNITGVEAENLLLTRGVDGSFLARPSKSNPGDFTLSVRRNGAVTHIKIQNTGDYYDLYGGEKFATLAKLVQYYMEHHGQLKEKNGDVIELKYPLNCADPTSERWFHGHLSGKEAEKLLTEKGKHGSFLVRESQSHPGDFVLSVRTGDDKGESNDGKSKVTHVMIRCQELKYDVGGGERFDSLTDLVEHYKKNPMVETLGTVLQLKQPLNTTRINAAEIESRVRELSKLAETTDKVKQGFWEEFETLQQQECKLLYSRKEGQRQENKNKNRYKNILPFDHTRVVLHDGDPNEPVSDYINANIIMPEFETKCNNSKPKKSYIATQGCLQNTVNDFWRMVFQENSRVIVMTTKEVERGKSKCVKYWPDEYALKEYGVMRVRNVKESAAHDYTLRELKLSKVGQGNTERTVWQYHFRTWPDHGVPSDPGGVLDFLEEVHHKQESIMDAGPVVVHCSAGIGRTGTFIVIDILIDIIREKGVDCDIDVPKTIQMVRSQRSGMVQTEAQYRFIYMAVQHYIETL'

In [24]:
pdbfile_path= './4DGP.pdb'
chain_id='A'
outpath='./dms_lib2'
dmspath = f'{outpath}/4DGP_dms_lib2.fasta'
seqfile_path = '/home/yunyao/SH2/4DGP.fasta'
write_dms_lib(
    argparse.Namespace(
        seqfile=seqfile_path,
        pdbfile=pdbfile_path,
        chain=chain_id,
        dmspath=dmspath,
        outpath=outpath
    )
)

In [27]:
# we will remove the first residue so that it matches experimental sequence
# for simplicity, we remove the G directly in fasta file
pdbfile_path= './6CRF.pdb'
chain_id='A'
outpath='./dms_lib2'
dmspath = f'{outpath}/6CRF_dms_lib2.fasta'
seqfile_path = '/home/yunyao/SH2/6CRF.fasta'
write_dms_lib(
    argparse.Namespace(
        seqfile=seqfile_path,
        pdbfile=pdbfile_path,
        chain=chain_id,
        dmspath=dmspath,
        outpath=outpath
    )
)

In [ ]:
# now we need to generate coordinates mask
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

# Function to align sequences and generate a mask
def align_sequences(seq1, seq2):
    # Perform global alignment using the Needleman-Wunsch algorithm
    alignments = pairwise2.align.globalxx(seq1, seq2)

    # Take the best alignment (highest score)
    best_alignment = alignments[0]
    aligned_seq1, aligned_seq2, score, start, end = best_alignment

    # Generate the mask
    mask = []
    for a, b in zip(aligned_seq1, aligned_seq2):
        if a == b and a != "-":
            mask.append(1)  # Match
        else:
            mask.append(0)  # Mismatch or gap

    return aligned_seq1, aligned_seq2, mask

# Example usage
full_4DGP = ''.join(read_fasta('/home/yunyao/SH2/4DGP.fasta').split())
full_6CRF = ''.join(read_fasta('/home/yunyao/SH2/6CRF.fasta').split())
Coor_seq_4DGP= get_native_seq('./4DGP.pdb', 'A')
Coor_seq_6CRF= get_native_seq('./6CRF.pdb', 'A')


aligned_seq1, aligned_seq2, mask = align_sequences(full_4DGP, Coor_seq_4DGP)

# Output the results
print("Aligned Sequence 1:", aligned_seq1)
print("Aligned Sequence 2:", aligned_seq2)
print("Mask:", mask)

# use the mask to generate new cooridnates
# when maks is 1, we keep the coordinates
# when mask is 0, we add np.inf to the coordinates
# maked_coords_4DGP shous have shape as (len(mask), 3, 3)
# when maask is 1, fill it with the coordinates
# when mask is 0, fill it with np.inf
import numpy as np  
# Assuming coords is a numpy array of shape (N, 3, 3) where N is the number of residues
fpath = './4DGP.pdb' # .pdb format is also acceptable
chain_id = 'A'
structure = esm.inverse_folding.util.load_structure(fpath, chain_id)
coords, native_seq = esm.inverse_folding.util.extract_coords_from_structure(structure)

#fill coords into masked_coords_4DGP
maksed_coords_4DGP=

np.where(np.array(mask)[:, None], coords, np.inf)
print('Masked coordinates shape:', maksed_coords_4DGP.shape)

Aligned Sequence 1: MTSRRWFHPNITGVEAENLLLTRGVDGSFLARPSKSNPGDFTLSVRRNGAVTHIKIQNTGDYYDLYGGEKFATLAELVQYYMEHHGQLKEKNGDVIELKYPLNCADPTSERWFHGHLSGKEAEKLLTEKGKHGSFLVRESQSHPGDFVLSVRTGDDKGESNDGKSKVTHVMIRCQELKYDVGGGERFDSLTDLVEHYKKNPMVETLGTVLQLKQPLNTTRINAAEIESRVRELSKLAETTDKVKQGFWEEFETLQQQECKLLYSRKEGQRQENKNKNRYKNILPFDHTRVVLHDGDPNEPVSDYINANIIMPEFETKCNNSKPKKSYIATQGCLQNTVNDFWRMVFQENSRVIVMTTKEVERGKSKCVKYWPDEYALKEYGVMRVRNVKESAAHDYTLRELKLSKVGQGNTERTVWQYHFRTWPDHGVPSDPGGVLDFLEEVHHKQESIMDAGPVVVHCSAGIGRTGTFIVIDILIDIIREKGVDCDIDVPKTIQMVRSQRSGMVQTEAQYRFIYMAVQHYIETLQRRLEHHHHHH
Aligned Sequence 2: --SRRWFHPNITGVEAENLLLTRGVDGSFLARPSKSNPGDFTLSVRRNGAVTHIKIQNTGDYYDLYGGEKFATLAELVQYYMEHHGQLKEKNGDVIELKYPLNCADPTSERWFHGHLSGKEAEKLLTEKGKHGSFLVRESQSHPGDFVLSVRTGDDKGESNDGKSKVTHVMIRCQELKYDVGGGERFDSLTDLVEHYKKNPMVETLGTVLQLKQPLNTTRINAAEIESRVRELS---------K-GFWEEFETLQQQECKLLYSRKEGQRQENKNKNRYKNILPFDHTRVVLH--D-----SDYINANIIMP--E--------KKSYIATQGCLQNTVNDFWRMVFQENSRVIVMTTKEVERGKSKCVKYWPDEYALKEYGVMRVRNVKESAAHDYTLRELKLSKVGQGNTERTVWQYHFRTW

ValueError: operands could not be broadcast together with shapes (536,1) (499,3,3) () 

In [18]:
  # now we can scan through the library and score each sequence
dms_lib = defaultdict(list)
with open(dmspath, 'r') as f:
    for line in f:
        if line.startswith('>'):
            seq_id = line.strip()[1:]  # remove '>'
        else:
            sequence = line.strip()
            dms_lib[seq_id].append(sequence)
# score each sequence in the library
scores = {}
for seq_id, sequences in dms_lib.items():
    for sequence in sequences:
        ll_fullseq, ll_withcoord = esm.inverse_folding.util.score_sequence(model, alphabet, coords, sequence)
        scores[seq_id] = (ll_fullseq, ll_withcoord)

In [ ]:
python ~/structural-evolution/bin/score_log_likelihoods.py ./4DGP.pdb  \
    ./dms_lib/4DGP_dms_lib.fasta --chain A \
    --outpath output/4DGP_mutated_seqs_scores.csv